In [ ]:
import pandas as pd
import holoviews as hv
import numpy as np
import geoviews as gv
import geoviews.feature as gf
import cartopy
import cartopy.feature as cf

from geoviews import opts
from cartopy import crs as ccrs

gv.extension('matplotlib', 'bokeh')

gv.output(dpi=120, fig='svg')

Cartopy and shapely make working with geometries and shapes very simple, and GeoViews provides convenient wrappers for the various geometry types they provide. In addition to Path and Polygons types, which draw geometries from lists of arrays or a geopandas DataFrame, GeoViews also provides the ``Feature`` and ``Shape`` types, which wrap cartopy Features and shapely geometries respectively.

### Feature

The Feature Element provides a very convenient means of overlaying a set of basic geographic features on top of or behind a plot. The ``cartopy.feature`` module provides various ways of loading custom features, however geoviews provides a number of default features which we have imported as ``gf``, amongst others this includes coastlines, country borders, and land masses. Here we demonstrate how we can plot these very easily, either in isolation or overlaid:

In [ ]:
(gf.ocean + gf.land + gf.ocean * gf.land * gf.coastline * gf.borders).cols(3)

These deafult features simply wrap around cartopy Features, therefore we can easily load a custom ``NaturalEarthFeature`` such as graticules at 30 degree intervals: 

In [ ]:
graticules = cf.NaturalEarthFeature(
    category='physical',
    name='graticules_30',
    scale='110m')

(gf.ocean() * gf.land() * gv.Feature(graticules, group='Lines') * gf.borders * gf.coastline).opts(
    opts.Feature('Lines', projection=ccrs.Robinson(), facecolor='none', edgecolor='gray'))

The scale of features may be controlled using the ``scale`` plot option, the most common options being `'10m'`, `'50m'` and `'110m'`. Cartopy will downloaded the requested resolution as needed.

In [ ]:
gv.output(backend='bokeh')

(gf.ocean * gf.land.options(scale='110m', global_extent=True) * gv.Feature(graticules, group='Lines') + 
 gf.ocean * gf.land.options(scale='50m', global_extent=True) * gv.Feature(graticules, group='Lines'))

Zoom in using the bokeh zoom widget and you should see that the right hand panel is using a higher resolution dataset for the land feature.

Instead of displaying a ``Feature`` directly it is also possible to request the geometries inside a ``Feature`` using the ``Feature.geoms`` method, which also allows specifying a ``scale`` and a ``bounds`` to select a subregion:

In [ ]:
gf.land.geoms('50m', bounds=(-10, 40, 10, 60))

When working interactively with higher resolution datasets it is sometimes necessary to dynamically update the geometries based on the current viewport. The ``resample_geometry`` operation is an efficient way to display only polygons that intersect with the current viewport and downsample polygons on-the-fly.

In [ ]:
gv.operation.resample_geometry(gf.coastline.geoms('10m')).opts(width=400, height=400, color='black')

Try zooming into the plot above and you will see the coastline geometry resolve to a higher resolution dynamically (this requires a live Python kernel).

### Shape

The ``gv.Shape`` object wraps around any shapely geometry, allowing finer grained control over each polygon. We can, for example, access the geometries on the ``LAND`` feature and display them individually. Here we will get the geometry corresponding to the Australian continent and display it using shapely's inbuilt SVG repr (not yet a HoloViews plot, just a bare SVG displayed by Jupyter directly):

In [ ]:
land_geoms = gf.land.geoms(as_element=False)
land_geoms[21]

Instead of letting shapely render it as an SVG, we can now wrap it in the ``gv.Shape`` object and let matplotlib or bokeh render it, alone or with other GeoViews or HoloViews objects:

In [ ]:
australia = gv.Shape(land_geoms[21])
alice_springs = gv.Text(133.870,-21.5, 'Alice Springs')

australia * gv.Points([(133.870,-23.700)]).opts(color='black', width=400) * alice_springs

We can also supply a list of geometries directly to a Polygons or Path element:

In [ ]:
gv.Polygons(land_geoms) + gv.Path(land_geoms)

This makes it possible to create choropleth maps, where each part of the geometry is assigned a value that will be used to color it. However, constructing a choropleth by combining a bunch of shapes can be a lot of effort and is error prone. For that reason, the Shape Element provides convenience methods to load geometries from a shapefile. Here we load the boundaries of UK electoral districts directly from an existing shapefile:

In [ ]:
hv.output(backend='matplotlib')

shapefile = '../assets/boundaries/boundaries.shp'
gv.Shape.from_shapefile(shapefile, crs=ccrs.PlateCarree())

To combine these shapes with some actual data, we have to be able to merge them with a dataset. To do so we can inspect the records the cartopy shapereader loads:

In [ ]:
shapes = cartopy.io.shapereader.Reader(shapefile)
list(shapes.records())[0]

As we can see, the record contains a ``MultiPolygon`` together with a standard geographic ``code``, which we can use to match up the geometries with a dataset. To continue we will require a dataset that is also indexed by these codes. For this purpose we load a dataset of the 2016 EU Referendum result in the UK:

In [ ]:
referendum = pd.read_csv('../assets/referendum.csv')
referendum = hv.Dataset(referendum)
referendum.data.head()

The ``from_records`` function optionally also supports merging the records and dataset directly. To merge them, supply the name of the shared attribute on which the merge is based via the ``on`` argument. If the name of attribute in the records and the dimension in the dataset match exactly, you can simply supply it as a string, otherwise supply a dictionary mapping between the attribute and column name. In this case we want to color the choropleth by the `'leaveVoteshare'`, which we define via the `value` argument.

Additionally we can request one or more indexes using the ``index`` argument. Finally we will declare the coordinate reference system in which this data is stored, which will in most cases be the simple Plate Carree projection.  We can then view the choropleth, with each shape colored by the specified value (the percentage who voted to leave the EU):

In [ ]:
hv.output(backend='bokeh')

gv.Shape.from_records(shapes.records(), referendum, on='code', value='leaveVoteshare',
                      index=['name', 'regionName']).opts(tools=['hover'], width=350, height=500)

## GeoPandas

GeoPandas extends the datatypes used by pandas to allow spatial operations on geometric types, which makes it a very convenient way of working with geometries with associated variables. GeoViews ``Path``, ``Contours`` and ``Polygons`` Elements natively support projecting and plotting of
geopandas DataFrames using both ``matplotlib`` and ``bokeh`` plotting extensions. We will load the example dataset of the world which also includes some additional data about each country:

In [ ]:
import geopandas as gpd
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.head()

We can simply pass the GeoPandas DataFrame to a Polygons, Path or Contours element and it will plot the data for us. The ``Contours`` and ``Polygons`` will automatically color the data by the first specified value dimension defined by the ``vdims`` keyword (the geometries may be colored by any dimension using the ``color_index`` plot option):

In [ ]:
gv.Polygons(world, vdims='pop_est').opts(projection=ccrs.Robinson(), width=600, tools=['hover'])

Geometries can be displayed using both matplotlib and bokeh, here we will switch to bokeh allowing us to color by a categorical variable (``continent``) and activating the hover tool to reveal information about the plot.

In [ ]:
gv.Polygons(world, vdims=['continent',  'name', 'pop_est']).opts(
    cmap='tab20', width=600, height=400, tools=['hover'], infer_projection=True)

The "Working with Bokeh" GeoViews notebook shows how to enable hover data that displays information about each of these shapes interactively.